In [1]:
import pandas as pd
import numpy as np
# Для импорта содержимого файлов tsv использую модуль pandas
# для оперирования массивами вещественных чисел применяю numpy

In [2]:
Hugo=pd.read_csv('Hugo.tsv', sep='\t') # Загружаю данные из Hugo
anno=pd.read_csv('Hugo_annotation.tsv', sep='\t') # Загружаю данные из аннотации Hugo
anno.loc[-1]=['SRR3184279', 'NR'] # исправляю, добавляю заголовок к аннотации
anno.index=anno.index+1
anno=anno.sort_index()
anno.rename(columns={'SRR3184279':'Patient','NR':'Response'}, inplace=True)
GSE=pd.read_csv('GSE91061.tsv', sep='\t') # Загружаю данных из GSE о значениях конц биомаркеров
GSE_anno=pd.read_csv('GSE91061_annotation.tsv', sep='\t') 
GSE_anno.loc[-1]=['SRR5088813', 'NR']
GSE_anno.index=GSE_anno.index+1
GSE_anno=GSE_anno.sort_index()
GSE_anno.rename(columns={'SRR5088813':'Patient','NR':'Response'}, inplace=True)
ibmrks=pd.read_csv('biomarkers_info.tsv', sep='\t') # Загружаю данные об ответе на лечение

In [3]:
# Удаляю из датасетов Hugo и GSE строки со значениями только NaN 
# и заменяю оставшиеся NaN на 0 
# заменяю булевы значения на 1 и 0
Hugo=Hugo.dropna(axis=0) # выбрасываю из датасета строки, не несущие информацию
Hugo=Hugo.fillna(0) # заменяю пустые значения на нули
Hugo=Hugo.replace({'true':1, 'false':0}) # заменяю булевы значения на численные эквиваленты
GSE=GSE.dropna(axis=0) # выбрасываю неинформативные строки
GSE=GSE.fillna(0) # заменяю на нули пустые значения
GSE=GSE.replace({'true':1, 'false':0}) # заменяю булевы значения на численные эквиваленты
bmrks=list(Hugo['Biomarker']) # получаю список биомаркеров

In [4]:
# цель данной функции - преобразовать данный список в список вещественных чисел
def list2float(L1):
    L2=[]
    for k in L1:
        if str(k)=='nan':
            L2.append(0)
        elif str(k)=='true':
            L2.append(1)
        elif str(k)=='false':
            L2.append(0)
        else:
            L2.append(float(k))
    return L2

In [5]:
# получаю список пациентов из датасета Hugo
patients=[]
for pat in Hugo.columns[1:]:
    patients.append(pat)

In [6]:
# функция должна получать верхний квартиль для данного numpy массива 
def get_top_Q(nparr):
    '''
    it obtains highest quartile of
    the numpy array
    '''
    q75=np.percentile(nparr,75)
    topQ_vals=[]
    topQ_vals_idxs=[]
    idx=0
    for b in nparr:
        if b>=q75 and b<=np.max(nparr):
            topQ_vals.append(b)
            topQ_vals_idxs.append(idx)
        idx+=1
    return topQ_vals, topQ_vals_idxs

In [7]:
# функция должна получать нижний квартиль для данного numpy массива 
def get_bottom_Q(nparr):
    '''
    it obtains highest quartile of
    the numpy array
    '''
    q25=np.percentile(nparr,25)
    bottomQ_vals=[]
    bottomQ_vals_idxs=[]
    idx=0
    for b in nparr:
        if b>=np.min(nparr) and b<=q25:
            bottomQ_vals.append(b)
            bottomQ_vals_idxs.append(idx)
        idx+=1
    return bottomQ_vals, bottomQ_vals_idxs

In [8]:
# получаю словарь конструкции "название маркера":"его направленность действия"
# чтобы было удобнее извлекать данные о маркерах
dict_ibmrks={}
for Id,Dir in zip(ibmrks['ID'].values, ibmrks['Direction'].values):
    dict_ibmrks[Id]=Dir

In [9]:
# словарь конструкции "название маркера":список его значений
# сделан для более удобного доступа к данным
dict_bmrks={}
# ibmrks_list=list(ibmrks['ID'])
for bm in bmrks:
    dict_bmrks[bm]=[]
    for p in Hugo.columns[1:]:
        i=bmrks.index(bm)
        if i in Hugo.index:
            dict_bmrks[bm].append(Hugo[p][bmrks.index(bm)])

In [10]:
# словарь соответствий между ID пациентов и откликом терапии
# из аннотации к Hugo
dict_anno={}
for k,v in zip(anno['Patient'].values, anno['Response'].values):
    dict_anno[k]=v

In [11]:
#  вычисляю валидности маркеров по следующему алгоритму:
#  для каждого маркера беру его направленность
#  если направленность - down тогда беру значения биомаркера
#  из нижней квартили, в противном случае - беру значения
#  из верхней квартили
# здесь произвожу вычисления для датасета Hugo

# получаю список названий маркеров
ibmrks_ids=list(ibmrks['ID'].values) # obtain list of ids from info about biomarkers table
# словарь соответствий между направлениями маркеров и функциями
dfuncs={'up':get_top_Q, 'down':get_bottom_Q}
ibmrks_validity={}
for bio in bmrks:
    eff=ibmrks['Direction'].values[ibmrks_ids.index(bio)] # get effect of the biomarker
    if bio in dict_bmrks.keys():
#        if len(dict_bmrks[bio])>0:
        marker_vals=np.array(list2float(dict_bmrks[bio]))
    if len(marker_vals)>0:
        tail, marker_vals_idxs=dfuncs[eff](marker_vals)
        score=0
        for idx in marker_vals_idxs:
            if dict_anno[patients[idx]]=='R':
                score+=1
        validity=float(score)/float(len(marker_vals_idxs))
        ibmrks_validity[bio]=round(validity,2)

In [12]:
ibmrks_validity

{'4c7f88d1-bc83-4fca-9ec8-051bc4a61da2': 0.43,
 'd2fc5540-652a-4f39-9295-a4ee8e567612': 0.43,
 '2b1ddac7-53dc-4b38-81b6-06f30d63e408': 0.57,
 'c365a4a5-5ce8-48a1-a1af-4b1b31bde9d9': 0.43,
 '668b6e8d-4d82-4024-8dc3-190a36408dac': 0.75,
 '8eed56c6-ae32-45b3-b5cf-c15556170c3f': 0.29,
 '6bba7cb6-0279-498f-91cc-f21beb1dbcbd': 0.57,
 'e61b6afe-6b0c-4e6c-8fa1-d7b069a8d135': 0.29,
 '6713e834-005b-48e5-9859-7f579c9bc137': 0.43,
 '599ef459-3c86-4a5e-a9f7-b539057ac9d8': 0.57,
 '2c64c0d9-cd01-44e5-83a5-a389095cd8d4': 0.71,
 'c7180d64-889d-49fc-b5dc-a142b9cce8bb': 0.43,
 'bf61e8e5-7ef6-4442-adc0-951486615ed5': 0.71,
 '6b9b1876-b4fb-45e4-ad73-65977cf3a6e2': 0.43,
 'ec9ac6f8-d919-4c85-9a88-2fa98839a187': 0.29,
 'de74cb41-14c0-48f4-9d43-824db35c5f17': 0.43,
 '1310bd8a-073a-4dca-ac14-5713421ea1ff': 0.57,
 '9af0bf92-ae69-4115-9db1-36c9b0cc5292': 0.57,
 'a51662c5-66d0-45da-85ea-a47f18c90263': 0.71,
 '60ba2f0a-933a-489d-8255-52d1c8ac93f5': 0.43,
 '68256091-8de9-4437-82e0-dcba5dea36df': 0.43,
 '59d836de-35

In [13]:
# словарь соответствий между ID пациентов и откликом терапии
# из аннотации к GSE
dict_anno_gse={}
for k,v in zip(GSE_anno['Patient'].values, GSE_anno['Response'].values):
    dict_anno_gse[k]=v

In [14]:
# получаю список пациентов из датасета GSE
patients_gse=[]
for pat in GSE.columns[1:]:
    patients_gse.append(pat)

In [15]:
# словарь конструкции "название маркера":список его значений
# сделан для более удобного доступа к данным
bmrks_gse=list(GSE['Biomarker']) # получаю список биомаркеров
dict_bmrks_gse={}
# ibmrks_list=list(ibmrks['ID'])
for bm in bmrks_gse:
    dict_bmrks_gse[bm]=[]
    for p in GSE.columns[1:]:
        i=bmrks_gse.index(bm)
        if i in GSE.index:
            dict_bmrks_gse[bm].append(GSE[p][bmrks_gse.index(bm)])

In [16]:
#  вычисляю валидности маркеров по следующему алгоритму:
#  для каждого маркера беру его направленность
#  если направленность - down тогда беру значения биомаркера
#  из нижней квартили, в противном случае - беру значения
#  из верхней квартили
# здесь произвожу вычисления для датасета GSE

ibmrks_validity_gse={}
for bio in bmrks_gse:
    eff=ibmrks['Direction'].values[ibmrks_ids.index(bio)] # get effect of the biomarker
    if bio in dict_bmrks_gse.keys():
        marker_vals=np.array(list2float(dict_bmrks_gse[bio]))
    if len(marker_vals)>0:
        tail,marker_vals_idxs=dfuncs[eff](marker_vals)
    score=0
    for idx in marker_vals_idxs:
        if patients_gse[idx] in dict_anno_gse.keys():
            if dict_anno_gse[patients_gse[idx]]=='R':
                score+=1
        validity=float(score)/float(len(marker_vals_idxs))
        ibmrks_validity_gse[bio]=round(validity,2)

In [17]:
# словарь со значениями валидностей маркеров, которые были вычислены
# из датасета Hugo
ibmrks_validity

{'4c7f88d1-bc83-4fca-9ec8-051bc4a61da2': 0.43,
 'd2fc5540-652a-4f39-9295-a4ee8e567612': 0.43,
 '2b1ddac7-53dc-4b38-81b6-06f30d63e408': 0.57,
 'c365a4a5-5ce8-48a1-a1af-4b1b31bde9d9': 0.43,
 '668b6e8d-4d82-4024-8dc3-190a36408dac': 0.75,
 '8eed56c6-ae32-45b3-b5cf-c15556170c3f': 0.29,
 '6bba7cb6-0279-498f-91cc-f21beb1dbcbd': 0.57,
 'e61b6afe-6b0c-4e6c-8fa1-d7b069a8d135': 0.29,
 '6713e834-005b-48e5-9859-7f579c9bc137': 0.43,
 '599ef459-3c86-4a5e-a9f7-b539057ac9d8': 0.57,
 '2c64c0d9-cd01-44e5-83a5-a389095cd8d4': 0.71,
 'c7180d64-889d-49fc-b5dc-a142b9cce8bb': 0.43,
 'bf61e8e5-7ef6-4442-adc0-951486615ed5': 0.71,
 '6b9b1876-b4fb-45e4-ad73-65977cf3a6e2': 0.43,
 'ec9ac6f8-d919-4c85-9a88-2fa98839a187': 0.29,
 'de74cb41-14c0-48f4-9d43-824db35c5f17': 0.43,
 '1310bd8a-073a-4dca-ac14-5713421ea1ff': 0.57,
 '9af0bf92-ae69-4115-9db1-36c9b0cc5292': 0.57,
 'a51662c5-66d0-45da-85ea-a47f18c90263': 0.71,
 '60ba2f0a-933a-489d-8255-52d1c8ac93f5': 0.43,
 '68256091-8de9-4437-82e0-dcba5dea36df': 0.43,
 '59d836de-35

In [18]:
# словарь со значениями валидностей маркеров, которые были вычислены
# из датасета GSE
ibmrks_validity_gse

{'74859816-44c7-48b6-92ad-772c31db9b07': 0.21,
 'c58f6f66-0a1c-49e7-a253-799f5ba5e39d': 0.14,
 'd2fc5540-652a-4f39-9295-a4ee8e567612': 0.36,
 'd8255301-4a7b-4cca-93ea-a58dfd11f28d': 0.18,
 'a12f6e6a-0cb5-4d2d-b54f-3228a8137fa5': 0.29,
 'c37d4319-f5b8-4d61-b6e4-8cc3c24ac36a': 0.18,
 '668b6e8d-4d82-4024-8dc3-190a36408dac': 0.28,
 '7ee6e4d3-ad63-4460-8139-a3964a988b5b': 0.25,
 '636941eb-00ca-4294-a845-cda68554667c': 0.21,
 '6bba7cb6-0279-498f-91cc-f21beb1dbcbd': 0.25,
 '230d2fec-8044-4125-b31d-ce14474c10ff': 0.18,
 '8e212732-bf20-4e93-9461-875c62d447a0': 0.18,
 'ddaea868-910c-481f-9d53-8f0229392ef0': 0.21,
 '599ef459-3c86-4a5e-a9f7-b539057ac9d8': 0.21,
 'c7180d64-889d-49fc-b5dc-a142b9cce8bb': 0.25,
 'bf61e8e5-7ef6-4442-adc0-951486615ed5': 0.14,
 '6b9b1876-b4fb-45e4-ad73-65977cf3a6e2': 0.25,
 '290c5968-cfcc-4020-b20f-d0070ba56880': 0.25,
 '7c4953d3-cd69-45ee-85bf-6052e33bcaa2': 0.18,
 '8b83bdb3-dbeb-467e-90ad-44bedcc31f8c': 0.25,
 'c677c455-2318-4823-ad1e-7256c6159b18': 0.18,
 '60ba2f0a-93

In [19]:
# обобщу полученные значения валидностей для биомаркеров,
# для этого возьму среднее арифметическое валидностей кажого биомаркера
# взяв их из обоих словарей
# ibmrks_validity - словарь валидностей маркеров, найденных из Hugo
# ibmrks_validity_gse - словарь валидностей, найденных из GSE
ibmrks_validity_general={}
for k in ibmrks_validity.keys():
    if k in ibmrks_validity_gse.keys():
        ibmrks_validity_general[k]=round((ibmrks_validity[k]+ibmrks_validity_gse[k])/2.0,2)

In [20]:
# обобщенные значения валидностей
ibmrks_validity_general

{'4c7f88d1-bc83-4fca-9ec8-051bc4a61da2': 0.38,
 'd2fc5540-652a-4f39-9295-a4ee8e567612': 0.4,
 '2b1ddac7-53dc-4b38-81b6-06f30d63e408': 0.41,
 'c365a4a5-5ce8-48a1-a1af-4b1b31bde9d9': 0.32,
 '668b6e8d-4d82-4024-8dc3-190a36408dac': 0.52,
 '8eed56c6-ae32-45b3-b5cf-c15556170c3f': 0.25,
 '6bba7cb6-0279-498f-91cc-f21beb1dbcbd': 0.41,
 '8e212732-bf20-4e93-9461-875c62d447a0': 0.44,
 '6713e834-005b-48e5-9859-7f579c9bc137': 0.3,
 '599ef459-3c86-4a5e-a9f7-b539057ac9d8': 0.39,
 'c7180d64-889d-49fc-b5dc-a142b9cce8bb': 0.34,
 'bf61e8e5-7ef6-4442-adc0-951486615ed5': 0.42,
 '6b9b1876-b4fb-45e4-ad73-65977cf3a6e2': 0.34,
 '290c5968-cfcc-4020-b20f-d0070ba56880': 0.41,
 'de74cb41-14c0-48f4-9d43-824db35c5f17': 0.3,
 '86bf4995-6890-4183-a5fc-6a8cecf82ace': 0.3,
 '1310bd8a-073a-4dca-ac14-5713421ea1ff': 0.41,
 '9af0bf92-ae69-4115-9db1-36c9b0cc5292': 0.39,
 '60ba2f0a-933a-489d-8255-52d1c8ac93f5': 0.34,
 '68256091-8de9-4437-82e0-dcba5dea36df': 0.3,
 '59d836de-35fe-4ed5-bcda-0579478d6181': 0.62,
 '5ef10d25-d068-4b